<a href="https://colab.research.google.com/github/Anastasiia-Ilina/Recommendation-systems/blob/main/Recommendation%20systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Получим данные

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies_ratings = movies.merge(ratings, on = 'movieId').reset_index(drop = True)
movies_ratings.dropna(inplace=True)
movies_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1.305696e+09


# Поработаем с гибридной моделью

In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808332 sha256=fa2a2569b1c0f8b8b3b660a07a93ceeba0cb634228c8daf94d5774c71f9c7b79
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357259 sha256=63a7352afd651bbcd2df5c622220f8945c212ccab087b79ec2125f119577b3b5
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from scipy import sparse
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import SVD

In [ ]:
def change_str(st):
   return ' '.join(st.replace(' ', '').replace('-', '').split('|'))

def recommend_for_user(user_id):
   #ищем фильмы, которые уже смотрели
   user_movies = movies_ratings[movies_ratings.userId == user_id].sort_values('timestamp').title.unique()

   #найдем последний просмотренный фильм
   last_user_movie = user_movies[-1]

   #сформируем словарь с жанрами и названиями фильмов
   title_genres = {}
   for index, row in movies.iterrows():
    title_genres[row.title] = row.genres

   #приведем список жанров в читабельный для модели вид
   movie_genre = title_genres[last_user_movie]
   movie_genre = change_str(movie_genre)

   #инициализируем и обучим первую моделью
   tfidf_transformer = TfidfTransformer()
   count_vect = CountVectorizer()
   movie_genres = [change_str(g) for g in movies.genres.values]
   X_train_counts = count_vect.fit_transform(movie_genres)
   X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

   neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean')
   neigh.fit(X_train_tfidf)

   #поработаем с первой моделью
   predict = count_vect.transform([movie_genre])
   X_tfidf = tfidf_transformer.transform(predict)
   res = neigh.kneighbors(X_tfidf, return_distance = True)

   movie_to_score  = movies.loc[res[1][0]].title.values

   #поработаем со второй моделью
   score = []
   title = []

   dataset = pd.DataFrame({
     'uid': movies_ratings.userId,
     'iid': movies_ratings.title,
     'rating': movies_ratings.rating
   })
   reader = Reader(rating_scale=(0.5, 5.0))
   data = Dataset.load_from_df(dataset, reader)
   trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
   algo = SVD(n_factors=60, n_epochs=20)
   algo.fit(trainset)

   for movie in movie_to_score:
     if movie in user_movies:
       continue
     score.append(algo.predict(uid = user_id, iid = movie).est)
     title.append(movie)

   #выведем полученные результаты
   best_indices = np.argsort(score)[-10:]
   for i in reversed(best_indices):
     print(title[i], round(score[i], 3))

In [ ]:
recommend_for_user(2.0)

Taxi Driver (1976) 4.149
Perfect World, A (1993) 4.021
Jackie Brown (1997) 4.02
Rope (1948) 3.978
Simple Plan, A (1998) 3.933
Fresh (1994) 3.893
Limey, The (1999) 3.878
Few Good Men, A (1992) 3.825
Lodger: A Story of the London Fog, The (1927) 3.819
Cape Fear (1962) 3.814


## Хочу поработать с моделью детальнее

In [ ]:
#подготовка данных
movie_genres = [change_str(g) for g in movies.genres.values]

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
title_genres = {}
for index, row in movies.iterrows():
  title_genres[row.title] = row.genres
last_user_movie = (movies_ratings[movies_ratings.userId == 2].sort_values('timestamp').title.unique())[-1]
movie_genre = change_str(title_genres[last_user_movie])

In [ ]:
dataset = pd.DataFrame({
       'uid': movies_ratings.userId,
       'iid': movies_ratings.title,
       'rating': movies_ratings.rating
})

In [ ]:
#теперь посмотрим модель
resalt = {'SVD_factors' : [],
          'SVD_epoch' : [],
          'RMSE' : []
}
neigh = NearestNeighbors(n_neighbors=20, metric='euclidean')
neigh.fit(X_train_tfidf)
predict = count_vect.transform([movie_genre])
X_tfidf = tfidf_transformer.transform(predict)
res = neigh.kneighbors(X_tfidf, return_distance = True)

movie_to_score  = movies.loc[res[1][0]].title.values

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
for ep in range(10, 30, 5):
  for k in range(50, 150, 10):
    algo = SVD(n_factors=k, n_epochs = ep)
    algo.fit(trainset)
    test_pred = algo.test(testset)
    resalt['RMSE'].append(accuracy.rmse(test_pred))
    resalt['SVD_factors'].append(k)
    resalt['SVD_epoch'].append(ep)

df_resalt = pd.DataFrame(resalt)
df_resalt.sort_values(by='RMSE', ascending=True).head(5)

RMSE: 0.8951
RMSE: 0.8951
RMSE: 0.8955
RMSE: 0.8942
RMSE: 0.8958
RMSE: 0.8966
RMSE: 0.8967
RMSE: 0.8972
RMSE: 0.8964
RMSE: 0.8987
RMSE: 0.8928
RMSE: 0.8927
RMSE: 0.8938
RMSE: 0.8933
RMSE: 0.8970
RMSE: 0.8970
RMSE: 0.8962
RMSE: 0.8996
RMSE: 0.8982
RMSE: 0.8987
RMSE: 0.8942
RMSE: 0.8909
RMSE: 0.8935
RMSE: 0.8996
RMSE: 0.8972
RMSE: 0.8937
RMSE: 0.8980
RMSE: 0.8971
RMSE: 0.8965
RMSE: 0.8997
RMSE: 0.8955
RMSE: 0.8957
RMSE: 0.8980
RMSE: 0.8965
RMSE: 0.8975
RMSE: 0.8964
RMSE: 0.8943
RMSE: 0.9019
RMSE: 0.8985
RMSE: 0.9020


,SVD_factors,SVD_epoch,RMSE
21,60,20,0.890932
11,60,15,0.892730
10,50,15,0.892764
13,80,15,0.893276
22,70,20,0.893459


# Поработаем с LightFM

In [ ]:
def transform_features(features, id_name):
  transform_feature = []
  for row in features.to_dict(orient = 'records'):
    id_values = row[id_name]
    del row[id_name]
    features_names = {key: value for key, value in row.items() if value != 0}
    transform_feature.append((id_values, features_names))

  return transform_feature

In [ ]:
from lightfm import LightFM
from lightfm.data import Dataset

from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k

In [ ]:
def light_recommend(user_id):

  tfidf = TfidfTransformer()
  count_vect = CountVectorizer()
  movie_genres = [change_str(g) for g in movies.genres.values]
  X_train_counts = count_vect.fit_transform(movie_genres)
  X_train_tfidf = tfidf.fit_transform(X_train_counts)

  features = tfidf.get_feature_names_out()
  tfidf_movie = pd.DataFrame(X_train_tfidf.toarray(), columns = features)
  tfidf_movie['movieId'] = movies['movieId']

  ds = Dataset()
  ds.fit(users = movies_ratings['userId'].unique(),
       items = movies['movieId'],
       item_features = features)

  item_features_matrix = ds.build_item_features(transform_features(tfidf_movie, 'movieId'))

  data = []
  for i, row in movies_ratings.iterrows():
    data.append((row['userId'], row['movieId']))
  interaction_matrix = ds.build_interactions(data=data)[0]

  model = LightFM(no_components = 30, random_state = 10, loss = 'logistic')
  model.fit(interaction_matrix, epochs = 10, item_features = item_features_matrix)

  n_items = interaction_matrix.shape[1]
  user_id = user_id
  scores = model.predict(user_id, np.arange(n_items))
  scores = pd.Series(scores)
  scores.index = movies['title']
  return scores.sort_values(ascending = False).head(5)

In [ ]:
light_recommend(2)

title
Schindler's List (1993)             1.647099
Silence of the Lambs, The (1991)    1.552240
Apollo 13 (1995)                    1.547294
Seven (a.k.a. Se7en) (1995)         1.541917
Shining, The (1980)                 1.516651
dtype: float32

## Хочу поработать с моделью детальнее

In [ ]:
#подготовка
tfidf = TfidfTransformer()
count_vect = CountVectorizer()
movie_genres = [change_str(g) for g in movies.genres.values]
X_train_counts = count_vect.fit_transform(movie_genres)
X_train_tfidf = tfidf.fit_transform(X_train_counts)

features = tfidf.get_feature_names_out()
tfidf_movie = pd.DataFrame(X_train_tfidf.toarray(), columns = features)
tfidf_movie['movieId'] = movies['movieId']

ds = Dataset()
ds.fit(users = movies_ratings['userId'].unique(),
       items = movies['movieId'],
       item_features = features)

item_features_matrix = ds.build_item_features(transform_features(tfidf_movie, 'movieId'))

data = []
for i, row in movies_ratings.iterrows():
  data.append((row['userId'], row['movieId']))
interaction_matrix = ds.build_interactions(data=data)[0]

In [ ]:
#поработаем с моделью
resalt_2 = {'no_components' : [],
          'epochs' : [],
          'recall' : [],
          'precision' : []
}

for comp in range(10, 100, 10):
  for ep in range(10, 50, 10):
      model = LightFM(no_components = comp, random_state = 10, loss = 'logistic')
      model.fit(interaction_matrix, epochs = ep, item_features = item_features_matrix)
      test_recall = recall_at_k(model, interaction_matrix, item_features = item_features_matrix).mean()
      test_precision = precision_at_k(model, interaction_matrix, item_features = item_features_matrix).mean()
      resalt_2['no_components'].append(comp)
      resalt_2['epochs'].append(ep)
      resalt_2['recall'].append(test_recall)
      resalt_2['precision'].append(test_precision)


In [ ]:
df_resalt_2 = pd.DataFrame(resalt_2)
df_resalt_2.sort_values(by='recall', ascending=False).head(5)

,no_components,epochs,recall,precision
3,10,40,0.002803,0.062500
7,20,40,0.002715,0.071875
11,30,40,0.002708,0.071528
23,60,40,0.002623,0.075694
15,40,40,0.002539,0.075000


In [ ]:
#сложно оценивать данные метрики, но получилось как получилось)

In [ ]:
n_items = interaction_matrix.shape[1]
user_id = 2
scores = model.predict(user_id, np.arange(n_items))
scores = pd.Series(scores)
scores.index = movies['title']
scores.sort_values(ascending = False).head(5)

title
Schindler's List (1993)             2.168383
Shining, The (1980)                 2.009567
Seven (a.k.a. Se7en) (1995)         2.006741
Apollo 13 (1995)                    1.988886
Silence of the Lambs, The (1991)    1.952184
dtype: float32